In [1]:
import cv2
import torch
import numpy as np
from diffusers import CogVideoXImageToVideoPipeline
from diffusers.utils import export_to_video, load_image

In [2]:
pipe = CogVideoXImageToVideoPipeline.from_pretrained(
    "THUDM/CogVideoX1.5-5B-I2V",
    torch_dtype=torch.bfloat16
)
pipe.enable_sequential_cpu_offload()
pipe.vae.enable_tiling()
pipe.vae.enable_slicing()

Loading pipeline components...:   0%|          | 0/5 [00:00<?, ?it/s]

Loading checkpoint shards:   0%|          | 0/4 [00:00<?, ?it/s]

In [3]:
pipe.load_lora_weights(
    "NimVideo/cogvideox1.5-5b-prompt-camera-motion",
    adapter_name="cogvideox-lora"
)
pipe.set_adapters(["cogvideox-lora"], [1.0])

In [4]:
height = 768  # 480, 768, 720, 1360
width = 1360
validation_image = '../resources/car.jpg'

image = load_image(validation_image).resize((width, height))
prompt = "Camera is moving to the left. A red sports car driving on a winding road."

In [5]:
video_generate = pipe(
    image=image,
    prompt=prompt,
    height=height, 
    width=width, 
    num_frames=81,
    guidance_scale=6,
    num_inference_steps=50,
    generator=torch.Generator(device="cuda").manual_seed(42),
).frames[0]

export_to_video(video_generate, 'out.mp4', fps=8)

  0%|          | 0/50 [00:00<?, ?it/s]

'out.mp4'